## requirements.txt를 사용한 패키지 설치

`!pip install -r requirements.txt`는 프로젝트에 필요한 Python 패키지들을 한 번에 설치하는 명령어입니다.

- **requirements.txt**: 설치할 패키지와 버전 정보가 기록된 파일.

In [10]:
# requirements.txt 파일에 명시된 패키지들을 설치
!pip install -r requirements.txt

  Using cached xmltodict-0.13.0-py2.py3-none-any.whl.metadata (7.7 kB)
  Using cached pandas-1.5.3.tar.gz (5.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: still running...
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


ERROR: Could not find a version that satisfies the requirement OpenDartReader==1.3.0 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.5, 0.0.6, 0.0.8, 0.0.9, 0.1.0, 0.1.1, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.2.0, 0.2.1, 0.2.2, 0.2.3)
ERROR: No matching distribution found for OpenDartReader==1.3.0


## 주요 라이브러리 임포트

### 1. `dotenv` 모듈
- **기능**: `.env` 파일에서 환경 변수를 읽어와 Python 환경에 설정합니다.
- **사용 예시**: API 키, 비밀 정보 등을 안전하게 관리.

### 2. `os` 모듈
- **기능**: 파일 경로, 환경 변수 등 운영 체제와 상호작용.

### 3. `requests` 라이브러리
- **기능**: HTTP 요청(예: GET, POST)을 보내고 서버 응답을 처리.

### 4. `io` 모듈
- **기능**: 메모리 파일 작업 지원. 예를 들어, 응답 데이터를 메모리에서 파일처럼 처리.

### 5. `zipfile` 모듈
- **기능**: ZIP 파일을 읽고 압축을 풀거나 생성.

### 6. `xmltodict` 라이브러리
- **기능**: XML 데이터를 Python의 딕셔너리로 변환하여 쉽게 조작 가능.

### 7. `pandas` 라이브러리
- **기능**: 데이터프레임 형식으로 데이터 분석 및 처리.


In [11]:
from dotenv import load_dotenv
import os
import requests
import io
import zipfile
import xmltodict
import pandas as pd 

## DART API 요청

- 환경 변수에서 API 키를 가져와 DART API에 GET 요청을 보냅니다.
- URL과 인증 키를 파라미터로 설정해 기업 고유번호 데이터를 요청합니다.


In [12]:
# .env 파일 로드
load_dotenv()

# 환경 변수에서 DART API 키를 가져옴
dart_api_key = os.environ.get("DART_API_KEY")

# DART API 요청 URL
url = "https://opendart.fss.or.kr/api/corpCode.xml"

# API 요청에 필요한 인증 키를 포함한 파라미터
params = {
    "crtfc_key": dart_api_key
}

# GET 요청을 통해 API 호출
resp = requests.get(url, params=params)

## 응답 데이터 처리 요약

- API 응답 내용을 메모리에서 파일처럼 처리하기 위해 `BytesIO` 객체로 변환합니다.  
- ZIP 파일로 압축된 데이터를 `zipfile.ZipFile`을 사용해 열어 처리할 준비를 합니다.


In [13]:
f = io.BytesIO(resp.content)
zfile = zipfile.ZipFile(f)

In [14]:
zfile.namelist()

['CORPCODE.xml']

## XML 데이터 처리 요약

- ZIP 파일에서 `CORPCODE.xml`을 읽고 UTF-8로 디코딩하여 XML 문자열을 얻습니다.  
- `xmltodict`를 사용해 XML 데이터를 Python 딕셔너리로 변환하고, 필요한 데이터를 추출하여 `pandas` DataFrame으로 변환합니다.


In [15]:
# ZIP 파일에서 CORPCODE.xml을 읽고 UTF-8로 디코딩
xml = zfile.read("CORPCODE.xml").decode("utf-8")

# XML 데이터를 딕셔너리로 변환
dict_data = xmltodict.parse(xml)

# 필요한 데이터 추출
data = dict_data['result']['list']

# DataFrame으로 변환
df = pd.DataFrame(data)

In [16]:
df.head()

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00430964,굿앤엘에스,None,20170630
2,00388953,크레디피아제이십오차유동화전문회사,None,20170630
3,00179984,연방건설산업,None,20170630
4,00420143,브룩스피알아이오토메이션잉크,None,20170630


- 'corp_name'이 '삼성전자'인 데이터를 필터링하여 출력합니다.


In [17]:
# 'corp_name'이 '삼성전자'인 데이터만 필터링
df[df['corp_name'] == '삼성전자']

,corp_code,corp_name,stock_code,modify_date
99632,00126380,삼성전자,005930,20240522


## DART API 공시 목록 조회 요약

- `OpenDartReader`를 사용해 삼성전자(005930)의 2022년부터 2024년까지 모든 공시 목록을 조회합니다.  
- `start`, `end` 날짜와 `kind='A'` 옵션을 사용하여 공시 목록을 필터링합니다.

In [18]:
import OpenDartReader

# 특정기업(삼성전자) 지난 3년간 모든 공시 목록
dart = OpenDartReader(dart_api_key)
dart.list('005930', start='2022-01-01', end='2024-12-30', kind='A')

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,분기보고서 (2024.09),20241114002642,삼성전자,20241114,
1,00126380,삼성전자,005930,Y,반기보고서 (2024.06),20240814003284,삼성전자,20240814,
2,00126380,삼성전자,005930,Y,분기보고서 (2024.03),20240516001421,삼성전자,20240516,
3,00126380,삼성전자,005930,Y,사업보고서 (2023.12),20240312000736,삼성전자,20240312,연
4,00126380,삼성전자,005930,Y,분기보고서 (2023.09),20231114002109,삼성전자,20231114,
5,00126380,삼성전자,005930,Y,반기보고서 (2023.06),20230814002534,삼성전자,20230814,
6,00126380,삼성전자,005930,Y,분기보고서 (2023.03),20230515002335,삼성전자,20230515,
7,00126380,삼성전자,005930,Y,사업보고서 (2022.12),20230307000542,삼성전자,20230307,연
8,00126380,삼성전자,005930,Y,분기보고서 (2022.09),20221114001832,삼성전자,20221114,
9,00126380,삼성전자,005930,Y,반기보고서 (2022.06),20220816001711,삼성전자,20220816,


## 공시 텍스트 추출 및 정리 요약

- `OpenDartReader`로 삼성전자의 공시 목록을 가져오고, 각 공시에서 '주요 제품 및 서비스', '주요 계약 및 연구개발활동', '기타 참고사항' 섹션을 추출합니다.  
- 추출한 텍스트를 데이터프레임의 새로운 컬럼에 저장하고, 공백을 정규 표현식으로 정리합니다.

In [19]:
from bs4 import BeautifulSoup
import re

# 삼성전자(005930)의 2022년부터 2024년까지 공시 목록 불러오기
df = dart.list('005930', start='2022-01-01', end='2024-12-30', kind='A')

# 섹션 내용 추출 함수
def extract_section_text(rcept_no):
    try:
        # 공시 XML 데이터 불러오기
        xml_data = dart.document(rcept_no)  
        soup = BeautifulSoup(xml_data, 'xml')
        
        # 'MANDATORY' 클래스의 'SECTION-2' 태그에서 정보 추출
        section_2 = soup.find_all('SECTION-2', {'ACLASS': 'MANDATORY'})
        
        results = {}
        for section in section_2:
            title = section.find('TITLE').text.strip() if section.find('TITLE') else 'No Title'
            text = section.get_text(separator=' ', strip=True) if section else 'No Content'
            # 관심 있는 섹션만 추출
            if title in ('2. 주요 제품 및 서비스', '6. 주요계약 및 연구개발활동', '7. 기타 참고사항'):
                results[title] = text
                
        # 섹션별 내용 반환
        return results.get('2. 주요 제품 및 서비스', 'No Data'), \
               results.get('6. 주요계약 및 연구개발활동', 'No Data'), \
               results.get('7. 기타 참고사항', 'No Data')
    
    except Exception as e:
        return 'Error', 'Error', 'Error'

# 섹션 데이터프레임에 추가
df[['주요 제품 및 서비스', '주요 계약 및 연구개발활동', '기타 참고사항']] = df['rcept_no'].apply(
    lambda x: pd.Series(extract_section_text(x))
)

# 모든 내용을 text 컬럼에 저장
df['text'] = df['주요 제품 및 서비스'] + df['주요 계약 및 연구개발활동'] + df['기타 참고사항']

# 정규 표현식을 사용하여 여러 공백을 하나의 공백으로 변환
def remove_extra_spaces(text):
    if isinstance(text, str):
        return re.sub(r'\s+', ' ', text)
    return text

# 불필요한 공백 제거
columns_to_clean = ['주요 제품 및 서비스', '주요 계약 및 연구개발활동', '기타 참고사항', 'text']
for col in columns_to_clean:
    df[col] = df[col].apply(remove_extra_spaces)

# 결과 확인
df


,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm,주요 제품 및 서비스,주요 계약 및 연구개발활동,기타 참고사항,text
0,00126380,삼성전자,005930,Y,분기보고서 (2024.09),20241114002642,삼성전자,20241114,,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."
1,00126380,삼성전자,005930,Y,반기보고서 (2024.06),20240814003284,삼성전자,20240814,,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."
2,00126380,삼성전자,005930,Y,분기보고서 (2024.03),20240516001421,삼성전자,20240516,,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."
3,00126380,삼성전자,005930,Y,사업보고서 (2023.12),20240312000736,삼성전자,20240312,연,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."
4,00126380,삼성전자,005930,Y,분기보고서 (2023.09),20231114002109,삼성전자,20231114,,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."
5,00126380,삼성전자,005930,Y,반기보고서 (2023.06),20230814002534,삼성전자,20230814,,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."
6,00126380,삼성전자,005930,Y,분기보고서 (2023.03),20230515002335,삼성전자,20230515,,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."
7,00126380,삼성전자,005930,Y,사업보고서 (2022.12),20230307000542,삼성전자,20230307,연,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."
8,00126380,삼성전자,005930,Y,분기보고서 (2022.09),20221114001832,삼성전자,20221114,,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."
9,00126380,삼성전자,005930,Y,반기보고서 (2022.06),20220816001711,삼성전자,20220816,,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에...",6. 주요계약 및 연구개발활동 가. 경영상의 주요 계약 등 계약 상대방 항 목 내 ...,7. 기타 참고사항 가. 지적재산권 관련 당사는 R 활동의 지적재산화에도 집중하여 ...,"2. 주요 제품 및 서비스 가. 주요 제품 매출 당사는 TV, 냉장고, 세탁기, 에..."


## 텍스트 결합 및 파일 저장 요약

- 데이터프레임의 `text` 컬럼에 있는 모든 텍스트를 하나로 결합하여 `all_text`에 저장합니다.  
- 결합된 텍스트를 UTF-8 인코딩으로 `output.txt` 파일에 저장합니다.

In [20]:
# 데이터프레임의 'text' 컬럼에 있는 모든 텍스트를 하나의 문자열로 결합
all_text = ''
for text in df.iloc:
    all_text += text['text']

# 결합된 텍스트를 txt 파일로 저장
with open("output.txt", "w", encoding="utf-8") as file:
    file.write(all_text)